In [1]:
import redis
import json

# Define connection variables
host = 'localhost'
port =  6379
password = None #'password'

# Connect to Redis
r = redis.Redis(host=host, port=port, password=password, decode_responses=True)
print('Connected to Redis')

r.flushdb()

Connected to Redis


True

# Identifying Top Customer Support Issues

## Description of the use case
This use case involves identifying the most common customer support issues to prioritize and address them effectively. By tracking and analyzing support tickets, banks can improve their customer service or build feature enhancements to address the concerns.

## Redis Data Structures Used
- **Hashes**: To store support ticket information.
- **Streams**: To capture and process real-time support ticket submissions.
- **Top-K**: To identify the most frequent support issues.

In [2]:
# Use Streams to capture real-time support ticket submissions
r.xadd('support_tickets', {'ticket_id': 'ticket_1001', 'issue': 'login_problem'})
r.xadd('support_tickets', {'ticket_id': 'ticket_1002', 'issue': 'payment_gway_issue'})
r.xadd('support_tickets', {'ticket_id': 'ticket_1003', 'issue': 'login_problem'})
r.xadd('support_tickets', {'ticket_id': 'ticket_1004', 'issue': 'ui_issue'})
r.xadd('support_tickets', {'ticket_id': 'ticket_1005', 'issue': 'payment_gway_issue'})
r.xadd('support_tickets', {'ticket_id': 'ticket_1006', 'issue': 'payment_gway_issue'})

# Initialize Top-K data structure
r.topk().reserve('top_issues', k=3, width=50, depth=7, decay=0.9)

# Function to process support tickets and update Top-K issues
def process_support_tickets():
    stream_entries = r.xrange('support_tickets')
    for entry in stream_entries:
        issue = entry[1]['issue']
        r.topk().add('top_issues', issue)

# Process the support tickets
process_support_tickets()

# Use Top-K to identify the most frequent support issues
top_issues = r.topk().list('top_issues')
print('Top Customer Support Issues:')
for issue in top_issues:
    print(issue)


Top Customer Support Issues:
payment_gway_issue
login_problem
ui_issue
